In [ ]:
import math
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from keras import layers
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import natsort
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import signal
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import neptune
from decouple import config
from keras.callbacks import Callback


In [ ]:
########################################################################################################################
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
print(device_lib.list_local_devices())
########################################################################################################################

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
########################################################################################################################
# Link to neptune ai for monitoring
########################################################################################################################
run = neptune.init(project_qualified_name='abdalraheem.ijjeh/aidd',
                   api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhZGJjNTcyNC0yN2ViLTQ5YzctOGFkZC1jODNlZmU1Y2Q4ZDcifQ==')
neptune.create_experiment('Variational_AutoEncoder_with_FFT')
neptune.append_tag('VAE_FFT')

In [ ]:
path = '/home/aijjeh/Desktop/Phd_Projects/Sequence_prediction/Sequence_to_sequence/Dataset_Full_wavefield_outputs_bottom'
os.chdir(path)

In [ ]:
Full_W_dataset = np.load('dataset_labels_Full_wavefield_475_512_512_512.npy', mmap_mode='r+')
print(Full_W_dataset.shape)

In [ ]:
path_cs_dataset = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/cs_datasets'
os.chdir(path_cs_dataset)

In [ ]:
CS_arr = np.load('CS_dataset_CR_50_percent.npy', mmap_mode='r+')

In [ ]:
print(CS_arr.shape)

In [ ]:
x_train = CS_arr[:304]
y_train = Full_W_dataset[:304]
x_val = CS_arr[304:380]
y_val = Full_W_dataset[304:380]
x_test = CS_arr[380:]
y_test = Full_W_dataset[380:]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
BATCH_SIZE = 1
train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
Input_img = tf.keras.layers.Input(shape=(512, 256, 256))  
    
#encoding architecture
x1 = tf.keras.layers.Conv2D(64,
            (3, 3),
            activation='relu',
            data_format='channels_first',
            padding='same',
            kernel_regularizer=regularizers.l1(10e-10))(Input_img)
x2 = tf.keras.layers.Conv2D(64,
            (3, 3), 
            activation='relu',
            padding='same',
            kernel_regularizer=regularizers.l1(10e-10))(x1)
x3 = tf.keras.layers.MaxPool2D(padding='same')(x2)

x4 = tf.keras.layers.Conv2D(128,
            (3, 3), 
            activation='relu', 
            padding='same',
            kernel_regularizer=regularizers.l1(10e-10))(x3)
x5 = tf.keras.layers.Conv2D(128,
            (3, 3),
            activation='relu',
            padding='same', 
            kernel_regularizer=regularizers.l1(10e-10))(x4)
x6 = tf.keras.layers.MaxPool2D(padding='same')(x5)
encoded = tf.keras.layers.Conv2D(256,
                 (3, 3), 
                 activation='relu', 
                 padding='same', 
                 kernel_regularizer=regularizers.l1(10e-10))(x6)
#encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)
# decoding architecture
x7 = tf.keras.layers.UpSampling2D()(encoded)
x8 = tf.keras.layers.Conv2D(128, 
            (3, 3), 
            activation='relu', 
            padding='same',
            kernel_regularizer=regularizers.l1(10e-10))(x7)
x9 = tf.keras.layers.Conv2D(128,
            (3, 3),
            activation='relu', 
            padding='same',
            kernel_regularizer=regularizers.l1(10e-10))(x8)
x10 = tf.keras.layers.Add()([x5, x9])
x11 = tf.keras.layers.UpSampling2D()(x10)
x12 = tf.keras.layers.Conv2D(64,
             (3, 3),
             activation='relu', 
             padding='same', 
             kernel_regularizer=regularizers.l1(10e-10))(x11)
x13 = tf.keras.layers.Conv2D(64, 
             (3, 3),
             activation='relu', 
             padding='same', 
             kernel_regularizer=regularizers.l1(10e-10))(x12)
x14 = tf.keras.layers.Add()([x2, x13])
# x3 = UpSampling2D((2, 2))(x3)
# x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
# x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
decoded = tf.keras.layers.Conv2D(512, 
                 (3, 3),
                 padding='same',
                 activation='relu', 
                 kernel_regularizer=regularizers.l1(10e-10))(x14)
autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])